In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
volume_dir = '/home/yuncong/CSHL_volumes/'
mesh_rootdir = create_if_not_exists('/home/yuncong/CSHL_meshes')

In [3]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [4]:
volume_landmark_names_unsided = ['12N', '5N', '6N', '7N', '7n', 'AP', 'Amb', 'LC',
                                 'LRt', 'Pn', 'R', 'RtTg', 'Tz', 'VLL', 'sp5']
linear_landmark_names_unsided = ['outerContour']

labels_unsided = volume_landmark_names_unsided + linear_landmark_names_unsided
labels_unsided_indices = dict((j, i+1) for i, j in enumerate(labels_unsided))  # BackG always 0

labelMap_unsidedToSided = {'12N': ['12N'],
                            '5N': ['5N_L', '5N_R'],
                            '6N': ['6N_L', '6N_R'],
                            '7N': ['7N_L', '7N_R'],
                            '7n': ['7n_L', '7n_R'],
                            'AP': ['AP'],
                            'Amb': ['Amb_L', 'Amb_R'],
                            'LC': ['LC_L', 'LC_R'],
                            'LRt': ['LRt_L', 'LRt_R'],
                            'Pn': ['Pn_L', 'Pn_R'],
                            'R': ['R_L', 'R_R'],
                            'RtTg': ['RtTg'],
                            'Tz': ['Tz_L', 'Tz_R'],
                            'VLL': ['VLL_L', 'VLL_R'],
                            'sp5': ['sp5'],
                           'outerContour': ['outerContour']}

labelMap_sidedToUnsided = {n: nu for nu, ns in labelMap_unsidedToSided.iteritems() for n in ns}

from itertools import chain
labels_sided = list(chain(*(labelMap_unsidedToSided[name_u] for name_u in labels_unsided)))
labels_sided_indices = dict((j, i+1) for i, j in enumerate(labels_sided)) # BackG always 0

In [5]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

In [6]:
stack = 'MD594'

In [7]:
# Load atlasProjected volume

atlasProjected_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlasProjected_polydata_list[name_s] = polydata

In [8]:
# Load localAdjusted volume

localAdjusted_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    localAdjusted_polydata_list[name_s] = polydata

In [9]:
# Load localAdjusted volume

centroid_localAdjusted_polydata_list = {}

for name_s, polydata in localAdjusted_polydata_list.iteritems():
    vertices, faces = polydata_to_mesh(polydata)
    centroid_localAdjusted_polydata_list[name_s] = vertices.mean(axis=0)

In [10]:
hessian_allLandmarks = pickle.load(open('/home/yuncong/CSHL_atlasAlignParams_atlas/hessian_allLandmarks.pkl', 'r'))
std_allLandmarks = pickle.load(open('/home/yuncong/CSHL_atlasAlignParams_atlas/std_allLandmarks.pkl', 'r'))

In [11]:
n_lm = len(hessian_allLandmarks)

length_factor = 1e6

R = defaultdict(dict)
V = defaultdict(dict)
S = defaultdict(dict)

for li, (name_s, hs) in enumerate(hessian_allLandmarks.iteritems()):

    for st, h in hs.iteritems():
        
        q = np.r_[3,7,11]
        h_tr = h[q[:,None], q]
        
        s, v = np.linalg.eigh(h_tr)
                
        S[name_s][st] = s
        V[name_s][st] = v
        
        R[name_s][st] = v*2e-9/s[None, :] * length_factor
        R[name_s][st] = R[name_s][st] * np.sign(R[name_s][st][0])
        
R.default_factory = None
V.default_factory = None
S.default_factory = None

In [12]:
anchor_points = [ centroid_localAdjusted_polydata_list[name_s] for name_s in available_labels_sided]
anchor_vectors2 = [ R[name_s][stack][:,2] for name_s in available_labels_sided]
anchor_vectors1 = [ R[name_s][stack][:,1] for name_s in available_labels_sided]
anchor_vectors0 = [ R[name_s][stack][:,0] for name_s in available_labels_sided]

In [13]:
# Show uncertainty ellipse

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)
    
for i, (name_s, polydata) in enumerate(localAdjusted_polydata_list.iteritems()):
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
    a.GetProperty().SetColor((1.,1.,1.))
    a.GetProperty().SetOpacity(.1)
    
    ren1.AddActor(a)
    
sphereSource = vtk.vtkSphereSource()

for p_ind in range(len(anchor_points)):
    
    anchor_point = anchor_points[p_ind]
    anchor_vector0 = anchor_vectors0[p_ind]
    anchor_vector1 = anchor_vectors1[p_ind]
    anchor_vector2 = anchor_vectors2[p_ind]
    
    length0 = np.linalg.norm(anchor_vector0)
    normalizedX = anchor_vector0/length0
    
    length1 = np.linalg.norm(anchor_vector1)
    normalizedY = anchor_vector1/length1
    
    length2 = np.linalg.norm(anchor_vector2)
    normalizedZ = anchor_vector2/length2
    
    matrix = vtk.vtkMatrix4x4()

    # Create the direction cosine matrix
    matrix.Identity()
    for i in range(3):
        matrix.SetElement(i, 0, normalizedX[i])
        matrix.SetElement(i, 1, normalizedY[i])
        matrix.SetElement(i, 2, normalizedZ[i])

    # Apply the transforms
    transform = vtk.vtkTransform()
    transform.Translate(anchor_point)
    transform.Concatenate(matrix)
    transform.Scale(length0, length1, length2)

    # Transform the polydata
    transformPD = vtk.vtkTransformPolyDataFilter()
    transformPD.SetTransform(transform)
    transformPD.SetInputConnection(sphereSource.GetOutputPort())

    #Create a mapper and actor for the arrow
    m = vtk.vtkPolyDataMapper()
    m.SetInputConnection(transformPD.GetOutputPort())
    
    a = vtk.vtkActor()
    a.SetMapper(m)
    
    a.GetProperty().SetRepresentationToWireframe()
    a.GetProperty().SetColor((1.,0.,0.))
#     a.GetProperty().SetOpacity(.3)

    ren1.AddActor(a)
    
    
# for name in ['LRt']:

#     score_volume = bp.unpack_ndarray_file(volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(name)s.bp' % \
#                                           {'stack':stack, 'name':name})
#     score_imagedata = volume_to_imagedata(score_volume)

#     volumeMapper = vtk.vtkSmartVolumeMapper()
#     #     volumeMapper.SetBlendModeToComposite()
#     volumeMapper.SetInputData(score_imagedata)

#     volumeProperty = vtk.vtkVolumeProperty()
#     #     volumeProperty.ShadeOff()
#     # volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

#     compositeOpacity = vtk.vtkPiecewiseFunction()
#     compositeOpacity.AddPoint(0.0, 0.0)
#     compositeOpacity.AddPoint(0.95, 0.01)
#     compositeOpacity.AddPoint(1.0, 1.0)
#     volumeProperty.SetScalarOpacity(compositeOpacity)

#     color = vtk.vtkColorTransferFunction()
# #     c = colors[name]
#     c = (1., 1., 1.)
#     color.AddRGBPoint(0.0, c[0], c[1], c[2])
#     color.AddRGBPoint(255.0, c[0], c[1], c[2])
#     volumeProperty.SetColor(color)

#     a = vtk.vtkVolume()
#     a.SetMapper(volumeMapper)
#     a.SetProperty(volumeProperty)
    
#     ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()

In [79]:
# Show three axes

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)
    
# for i, (name_s, polydata) in enumerate(localAdjusted_polydata_list.iteritems()):
    
#     m = vtk.vtkPolyDataMapper()
#     m.SetInputData(polydata)

#     a = vtk.vtkActor()
#     a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
# #     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
#     a.GetProperty().SetColor((1.,1.,1.))
#     a.GetProperty().SetOpacity(.1)
    
#     ren1.AddActor(a)
    
    
arrowSource = vtk.vtkArrowSource()

for anchor_vectors, c in zip([anchor_vectors2, anchor_vectors1, anchor_vectors0],
                            [(1.,0.,0.),(0.,1.,0.),(0.,0.,1.)]):

    for p_ind in range(len(anchor_points)):

        anchor_point = anchor_points[p_ind]
        anchor_vector = anchor_vectors[p_ind]

        length = np.linalg.norm(anchor_vector)
        normalizedX = anchor_vector/length

        arbitrary = np.random.uniform(-10, 10, 3)
        normalizedZ = np.cross(normalizedX, arbitrary)
        normalizedZ = normalizedZ/np.linalg.norm(normalizedZ)
        normalizedY = np.cross(normalizedZ, normalizedX)
        normalizedY = normalizedY/np.linalg.norm(normalizedY)

        matrix = vtk.vtkMatrix4x4()

        # Create the direction cosine matrix
        matrix.Identity()
        for i in range(3):
            matrix.SetElement(i, 0, normalizedX[i])
            matrix.SetElement(i, 1, normalizedY[i])
            matrix.SetElement(i, 2, normalizedZ[i])

        # Apply the transforms
        transform = vtk.vtkTransform()
        transform.Translate(anchor_point)
        transform.Concatenate(matrix)
        transform.Scale(length, length, length)

        # Transform the polydata
        transformPD = vtk.vtkTransformPolyDataFilter()
        transformPD.SetTransform(transform)
        transformPD.SetInputConnection(arrowSource.GetOutputPort())

        #Create a mapper and actor for the arrow
        m = vtk.vtkPolyDataMapper()
        m.SetInputConnection(transformPD.GetOutputPort())

        a = vtk.vtkActor()
        a.SetMapper(m)

        a.GetProperty().SetColor(c)

        ren1.AddActor(a)
    
    
# for name in ['Pn']:

#     score_volume = bp.unpack_ndarray_file(volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(name)s.bp' % \
#                                           {'stack':stack, 'name':name})
#     score_imagedata = volume_to_imagedata(score_volume)

#     volumeMapper = vtk.vtkSmartVolumeMapper()
#     #     volumeMapper.SetBlendModeToComposite()
#     volumeMapper.SetInputData(score_imagedata)

#     volumeProperty = vtk.vtkVolumeProperty()
#     #     volumeProperty.ShadeOff()
#     # volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

#     compositeOpacity = vtk.vtkPiecewiseFunction()
#     compositeOpacity.AddPoint(0.0, 0.0)
#     compositeOpacity.AddPoint(0.95, 0.01)
#     compositeOpacity.AddPoint(1.0, 1.0)
#     volumeProperty.SetScalarOpacity(compositeOpacity)

#     color = vtk.vtkColorTransferFunction()
# #     c = colors[name]
#     c = (1., 1., 1.)
#     color.AddRGBPoint(0.0, c[0], c[1], c[2])
#     color.AddRGBPoint(255.0, c[0], c[1], c[2])
#     volumeProperty.SetColor(color)

#     a = vtk.vtkVolume()
#     a.SetMapper(volumeMapper)
#     a.SetProperty(volumeProperty)
    
#     ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()

In [ ]:
# Show three axes, glyph3D approach

vector_map = np.zeros((10,10,10,3))
for p_ind in range(len(anchor_points)):
    x,y,z = anchor_points[p_ind]
    vector_map[y,x,z] = anchor_vectors[p_ind]
    
arrowSource = vtk.vtkArrowSource()
arrowSource.Update()

glyphFilter = vtk.vtkGlyph3D()
glyphFilter.SetSourceConnection(arrowSource.GetOutputPort())
glyphFilter.OrientOn()
glyphFilter.SetVectorModeToUseVector()
glyphFilter.SetScaleModeToScaleByVector()
# glyphFilter.SetScaleModeToDataScalingOff()
glyphFilter.SetColorModeToColorByScalar()

imagedata = vectormap_to_imagedata(vector_map, colors=[255,255,255])
glyphFilter.SetInputData(imagedata)
glyphFilter.Update()

m = vtk.vtkPolyDataMapper()
m.SetInputConnection(glyphFilter.GetOutputPort())
m.ScalarVisibilityOn()

ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)


# m = vtk.vtkPolyDataMapper()
# m.SetInputData(polydata)

a = vtk.vtkActor()
a.SetMapper(m)
# a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
# a.GetProperty().SetColor((1.,1.,1.))

ren1.AddActor(a)

ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()